In [8]:
# Configure plots for inline use in Jupyter Notebook
%matplotlib inline

import datetime as dt

# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
)

In [9]:
# set credentials for Sentinel Hub and AWS
import os
config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")

# Initialize SentinelHubBYOC class
byoc = SentinelHubBYOC(config=config)

In [32]:
# list collections and tiles
collections_iterator = byoc.iter_collections()
my_collections = list(collections_iterator)

for collection in my_collections:
    print((collection["name"], collection["id"]))
    print(collection["additionalData"])
    tiles = list(byoc.iter_tiles(collection))
    # for tile in tiles:
    #     print(tile['status'])
    #     print(tile['created'])
    #     print(tile['additionalData'])
    print("-------------------")

('urban_audit_2021_city', '20774d38-7b2e-43e9-b574-66fa2954da1c')
{'bands': {'B1': {'bitDepth': 16, 'source': 'B1', 'bandIndex': 1, 'sampleFormat': 'UINT'}}}
-------------------
('UrbanAtlas2018_10m_raster', '55c49191-4afa-4384-956e-e9b746f259f3')
{'bands': {'B1': {'bitDepth': 16, 'source': 'B1', 'bandIndex': 1, 'sampleFormat': 'UINT'}}, 'maxMetersPerPixel': 6400.0, 'extent': {'type': 'Polygon', 'coordinates': [[[-49.396508783, 26.176870268], [-49.396508783, 70.57381085], [67.130764957, 70.57381085], [67.130764957, 26.176870268], [-49.396508783, 26.176870268]]]}, 'hasSensingTimes': 'NO'}
-------------------
('ImperviousnessDensity2018_10m_raster', '94c0149f-131a-4c0c-8809-11aeab5da253')
{'bands': {'B01': {'bitDepth': 8, 'source': 'B01', 'bandIndex': 1, 'sampleFormat': 'UINT'}}, 'maxMetersPerPixel': 6400.0, 'extent': {'type': 'Polygon', 'coordinates': [[[-56.505141902, 24.284177011], [-56.505141902, 72.664408538], [72.906136759, 72.664408538], [72.906136759, 24.284177011], [-56.50514190

In [31]:
# delete unused collections
# collection_del = my_collections[2]
# print(collection_del)
# byoc.delete_collection(collection_del)

{'id': '81a17892-38ff-470a-ae46-b64313acd7b3', 'userId': 'a6e8d88e-0eed-4bd1-bb87-9d1bfbb204ff', 'accountId': 'd4403346-93d0-4ea6-989b-d7a990989ca6', 'name': 'nuts3_2021', 's3Bucket': 'hub-fairicube0', 'additionalData': {'bands': {'B1': {'bitDepth': 16, 'source': 'B1', 'bandIndex': 1, 'sampleFormat': 'UINT'}}, 'maxMetersPerPixel': 8000.0, 'extent': {'type': 'Polygon', 'coordinates': [[[-70.162550554, 24.802797666], [-70.162550554, 80.971144094], [87.127601697, 80.971144094], [87.127601697, 24.802797666], [-70.162550554, 24.802797666]]]}, 'hasSensingTimes': 'NO'}, 'noData': 65535, 'created': '2023-03-07T14:15:11.719334Z', 'requiresMetadataUpdate': False, 'isConfigured': True}


''

In [27]:
# rename collections
# ua2018 = my_collections[1]
# ua2018["name"] = "UrbanAtlas2018_10m_raster"
byoc.update_collection(ua2018)

''

In [29]:
ua2018

{'id': '55c49191-4afa-4384-956e-e9b746f259f3',
 'userId': 'a6e8d88e-0eed-4bd1-bb87-9d1bfbb204ff',
 'accountId': 'd4403346-93d0-4ea6-989b-d7a990989ca6',
 'name': 'UrbanAtlas2018_10m_raster',
 's3Bucket': 'hub-fairicube0',
 'additionalData': {'bands': {'B1': {'bitDepth': 16,
    'source': 'B1',
    'bandIndex': 1,
    'sampleFormat': 'UINT'}},
  'maxMetersPerPixel': 6400.0,
  'extent': {'type': 'Polygon',
   'coordinates': [[[-49.396508783, 26.176870268],
     [-49.396508783, 70.57381085],
     [67.130764957, 70.57381085],
     [67.130764957, 26.176870268],
     [-49.396508783, 26.176870268]]]},
  'hasSensingTimes': 'NO'},
 'noData': 65535,
 'created': '2023-05-15T08:39:27.543561Z',
 'requiresMetadataUpdate': False,
 'isConfigured': True}

In [7]:
collections = catalog.get_collections()

collections = [collection for collection in collections if collection["id"].startswith(("byoc", "batch"))]

collections

[{'stac_version': '1.0.0',
  'stac_extensions': [],
  'type': 'Collection',
  'id': 'byoc-20774d38-7b2e-43e9-b574-66fa2954da1c',
  'title': 'urban_audit_2021_city',
  'description': 'urban_audit_2021_city',
  'license': 'proprietary',
  'keywords': [],
  'providers': [],
  'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
   'temporal': {'interval': [[None, None]]}},
  'summaries': {},
  'links': [{'href': 'https://services.sentinel-hub.com/api/v1/catalog/1.0.0/',
    'rel': 'root',
    'type': 'application/json'},
   {'href': 'https://services.sentinel-hub.com/api/v1/catalog/1.0.0/collections/byoc-20774d38-7b2e-43e9-b574-66fa2954da1c',
    'rel': 'self',
    'type': 'application/json'},
   {'href': 'https://services.sentinel-hub.com/api/v1/catalog/1.0.0/collections',
    'rel': 'parent',
    'type': 'application/json'},
   {'href': 'https://services.sentinel-hub.com/api/v1/catalog/1.0.0/collections/byoc-20774d38-7b2e-43e9-b574-66fa2954da1c/queryables',
    'rel': 'http:/